In [1]:

import spacy
from spacy import displacy
import re
import spacy
from spacy import displacy
contador_llamadas=0
nlp = spacy.load("es_core_news_sm")

def mostrar_arboles(text):
    global contador_llamadas
    """
    Muestra el árbol sintáctico de la frase original y luego el de la frase
    sin preposiciones, artículos, conjunciones y pronombres.

    Parámetros:
    - text (str): La frase de entrada.

    Retorno:
    - str: La frase sin preposiciones, artículos, conjunciones y pronombres.
    """
    # Analizar la frase original
    doc = nlp(text)


    # Filtrar palabras que no son preposiciones, artículos, conjunciones ni pronombres
    filtered_words = [
        token for token in doc
        if token.pos_ not in {"ADP", "DET", "CCONJ", "SCONJ", "PRON"}
    ]
    # Reconstruir la frase sin estas palabras
    filtered_sentence = " ".join([token.text for token in filtered_words])
    # Crear un nuevo Doc a partir de las palabras filtradas
    filtered_sentence = re.sub(r'\S*@\S*\s?', '', filtered_sentence)
    contador_llamadas += 1
    print(contador_llamadas)

    return filtered_sentence

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import classification_report
import numpy as np

# Nombre de carpeta y archivo
nombredecarpetaatrabajar = "REST_MEX/"
nombredearchivo = "Rest_Mex_Sentiment_Analysis_2023_Train.xlsx"
nombre_archivo = 'CORPUS/' + nombredecarpetaatrabajar + nombredearchivo

# Cargar datos desde el archivo CSV
full_data = pd.read_excel(nombre_archivo)  # Usamos read_csv porque es un archivo CSV
print(full_data)
# Convertir todos los valores de 'content' a string
full_data['lemma'] = full_data['TituloyResena'].astype(str)






In [ ]:

# Contar filas antes de eliminar NaN
antes = len(full_data)

# Reemplazar los valores en la columna 'Polarity'
full_data['Polarity'] = full_data['Polarity'].replace({1: 0, 2: 0, 3 : np.nan, 4: 1, 5: 1})

# Eliminar filas con NaN en la columna 'Polarity'
full_data.dropna(subset=['Polarity'], inplace=True)

print(full_data)

# Contar filas después de eliminar NaN
despues = len(full_data)

print("Número de filas antes de eliminar NaN:", antes)
print("Número de filas después de eliminar NaN:", despues)


In [3]:
full_data.to_pickle('CORPUS/'+nombredecarpetaatrabajar+'/Dataset.pkl')
full_data = pd.read_pickle('CORPUS/'+nombredecarpetaatrabajar+'/Dataset.pkl')

In [4]:
import torch

print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Nombre del dispositivo CUDA:", torch.cuda.get_device_name(0))
    print("Versión de CUDA:", torch.version.cuda)

CUDA disponible: True
Nombre del dispositivo CUDA: NVIDIA GeForce RTX 2060 SUPER
Versión de CUDA: 11.8


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
try:
    tensor = torch.tensor([1.0, 2.0, 3.0]).to(device)
    print("Tensor en dispositivo:", tensor.device)
except Exception as e:
    print("Error al mover tensor a la GPU:", e)

Tensor en dispositivo: cuda:0


In [6]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import classification_report
import numpy as np

# Definir nombres de archivo
nombredecarpetaatrabajar = "REST_MEX/"
nombredearchivo = "Rest_Mex_Sentiment_Analysis_2023_Train"
nombre_archivo = 'CORPUS/' + nombredecarpetaatrabajar + nombredearchivo
print(nombre_archivo)

# Cargar el DataFrame
full_data =  pd.read_pickle('CORPUS/'+nombredecarpetaatrabajar+'/Dataset.pkl')
print(full_data)

# Seleccionar las columnas necesarias
data = full_data[['lemma', 'sentimiento']]
data['sentimiento'] = data['sentimiento'].astype(int)  # Asegurarse de que las etiquetas sean enteras

unique_labels = data['sentimiento'].unique()
print("Etiquetas únicas:", unique_labels)
print("Número de etiquetas únicas:", len(unique_labels))

# Dividir el DataFrame en conjunto de entrenamiento (80%) y prueba (20%)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Verificar si CUDA está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Cargar el tokenizador y el modelo BETO
model_name = "dccuchile/bert-base-spanish-wwm-cased"  # BETO
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Cambia num_labels según tus clases

# Mover el modelo a la GPU si está disponible
model.to(device)

# Convertir los DataFrames a conjuntos de full_data de Hugging Face
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Tokenización
def tokenize_function(examples):
    tokenized = tokenizer(examples['lemma'], truncation=True, padding='max_length', max_length=256)
    tokenized['labels'] = examples['sentimiento']  # Agregar etiquetas
    return tokenized

# Aplicar tokenización
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Configuración de los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Reducido para ahorrar memoria
    per_device_eval_batch_size=8,    # Reducido para ahorrar memoria
    num_train_epochs=5,               # Ajustar según necesidad
    weight_decay=0.01,
    fp16=True,                        # Habilitar FP16 para ahorro de memoria
)

# Definir el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

# Entrenar el modelo
trainer.train()

# Hacer predicciones en el conjunto de prueba
predictions = trainer.predict(tokenized_test)
predicted_classes = np.argmax(predictions.predictions, axis=1)

# Convertir las etiquetas verdaderas a un formato numpy
y_test_np = np.array(test_data['sentimiento'])

# Generar el informe de clasificación
report = classification_report(y_test_np, predicted_classes, zero_division=0)
print(report)

# Guardar el modelo
output_dir = nombredecarpetaatrabajar+"/constopwords/"+"/model"
os.makedirs(output_dir, exist_ok=True)  # Crear el directorio si no existe
print("Guardando el modelo y el tokenizador en:", output_dir)
model.save_pretrained(output_dir, safe_serialization=False)
tokenizer.save_pretrained(output_dir)
print("Modelo y tokenizador guardados exitosamente.")

# Cargar el modelo y el tokenizador para verificar
print("Cargando el modelo y el tokenizador desde:", output_dir)
loaded_tokenizer = BertTokenizer.from_pretrained(output_dir)
loaded_model = BertForSequenceClassification.from_pretrained(output_dir)

# Verificar si CUDA está disponible y mover el modelo a la GPU si es posible
loaded_model.to(device)

# Función para predecir el sentimiento de un texto
def predict_sentiment(text):
    inputs = loaded_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Mover los inputs a la GPU
    with torch.no_grad():  # Desactivar el cálculo de gradientes
        outputs = loaded_model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()
    return predicted_class

# Probar con un ejemplo
text_example = "Este producto es excelente!"
predicted_class = predict_sentiment(text_example)
print(f"Texto: {text_example} -> Clase Predicha: {predicted_class}")


CORPUS/IMDB/IMDBDatasetSPANISH.csv
       Unnamed: 0                                          review_en  \
0               0  One of the other reviewers has mentioned that ...   
1               1  A wonderful little production. The filming tec...   
2               2  I thought this was a wonderful way to spend ti...   
3               3  Basically there's a family where a little boy ...   
4               4  Petter Mattei's "Love in the Time of Money" is...   
...           ...                                                ...   
49995       49995  I thought this movie did a down right good job...   
49996       49996  Bad plot, bad dialogue, bad acting, idiotic di...   
49997       49997  I am a Catholic taught in parochial elementary...   
49998       49998  I'm going to have to disagree with the previou...   
49999       49999  No one expects the Star Trek movies to be high...   

                                               review_es sentiment  \
0      Uno de los otros crític

C:\Users\GpJonat\AppData\Local\Temp\ipykernel_17616\567952050.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentimiento'] = data['sentimiento'].astype(int)  # Asegurarse de que las etiquetas sean enteras


Etiquetas únicas: [1 0]
Número de etiquetas únicas: 2
cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

...

{'eval_loss': 0.5410951375961304, 'eval_runtime': 34.1146, 'eval_samples_per_second': 293.13, 'eval_steps_per_second': 36.641, 'epoch': 4.0}
{'loss': 0.0223, 'grad_norm': 0.12309235334396362, 'learning_rate': 3.6088e-06, 'epoch': 4.1}
{'loss': 0.0179, 'grad_norm': 0.0036281610373407602, 'learning_rate': 3.2096000000000006e-06, 'epoch': 4.2}
{'loss': 0.017, 'grad_norm': 0.004299131687730551, 'learning_rate': 2.8096e-06, 'epoch': 4.3}
{'loss': 0.0284, 'grad_norm': 0.004546491429209709, 'learning_rate': 2.4096e-06, 'epoch': 4.4}
{'loss': 0.0249, 'grad_norm': 0.07431141287088394, 'learning_rate': 2.0096e-06, 'epoch': 4.5}
{'loss': 0.0235, 'grad_norm': 0.00965035054832697, 'learning_rate': 1.6096e-06, 'epoch': 4.6}
{'loss': 0.0298, 'grad_norm': 0.004322187975049019, 'learning_rate': 1.2096e-06, 'epoch': 4.7}
{'loss': 0.0158, 'grad_norm': 0.0035967326257377863, 'learning_rate': 8.096000000000002e-07, 'epoch': 4.8}
{'loss': 0.0193, 'grad_norm': 0.0032111613545566797, 'learning_rate': 4.096000

  0%|          | 0/1250 [00:00<?, ?it/s]

{'eval_loss': 0.632770299911499, 'eval_runtime': 34.001, 'eval_samples_per_second': 294.109, 'eval_steps_per_second': 36.764, 'epoch': 5.0}
{'train_runtime': 3256.703, 'train_samples_per_second': 61.412, 'train_steps_per_second': 7.676, 'train_loss': 0.15042834924697876, 'epoch': 5.0}


  0%|          | 0/1250 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      4961
           1       0.92      0.91      0.92      5039

    accuracy                           0.92     10000
   macro avg       0.91      0.92      0.91     10000
weighted avg       0.92      0.92      0.92     10000

Guardando el modelo y el tokenizador en: IMDB//constopwords//model
Modelo y tokenizador guardados exitosamente.
Cargando el modelo y el tokenizador desde: IMDB//constopwords//model
Texto: Este producto es excelente! -> Clase Predicha: 1


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Generar la matriz de confusión
conf_matrix = confusion_matrix(y_test_np, predicted_classes)

# Etiquetas personalizadas
labels_x = ["Falsos Positivos (FV)", "Verdaderos Positivos (VV)"]
labels_y = ["Falsos Negativos (VF)", "Verdaderos Negativos (FF)"]

# Visualizar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels_x, yticklabels=labels_y)
plt.title('Matriz de Confusión con Etiquetas Descriptivas')
plt.xlabel('Predicciones')
plt.ylabel('Valores Reales')
plt.show()

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import classification_report
import numpy as np
from lime.lime_text import LimeTextExplainer

# Cargar el modelo y el tokenizador para verificar
nombredecarpetaatrabajar = "IMDB/"
nombredearchivo = "IMDBDatasetSPANISH.csv"
nombre_archivo = 'CORPUS/' + nombredecarpetaatrabajar + nombredearchivo
output_dir = nombredecarpetaatrabajar + "/constopwords/" + "/model"
print("Cargando el modelo y el tokenizador desde:", output_dir)
loaded_tokenizer = BertTokenizer.from_pretrained(output_dir)
loaded_model = BertForSequenceClassification.from_pretrained(output_dir)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Verificar si CUDA está disponible y mover el modelo a la GPU si es posible
loaded_model.to(device)

# Función para predecir el sentimiento de un texto
def predict_sentiment(text):
    inputs = loaded_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Mover los inputs a la GPU
    with torch.no_grad():  # Desactivar el cálculo de gradientes
        outputs = loaded_model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()
    return predicted_class

# Modificar la función para devolver probabilidades para LIME
def predict_proba(texts):
    inputs = loaded_tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Mover los inputs a la GPU
    with torch.no_grad():
        outputs = loaded_model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1).cpu().numpy()  # Convertir a numpy para LIME
    return probabilities

# Probar con un ejemplo
text_example = "Es fantástico cuando una película comica no te hace reir.¡¡Qué pena!!Esta película es muy aburrida y larga.Es simplemente doloroso.La historia es da pena y no es divertido. Te sientes mejor cuando está termina"
predicted_class = predict_sentiment(text_example)
print(f"Texto: {text_example} -> Clase Predicha: {predicted_class}")

# Generar explicación con LIME
class_names = ["Negativo", "Positivo"]  # Cambiar según las clases de tu modelo
explainer = LimeTextExplainer(class_names=class_names)

# Crear explicación para un texto
exp = explainer.explain_instance(
    text_example,                # Texto de ejemplo
    predict_proba,               # Función de predicción
    num_features=10,             # Número de palabras más importantes a mostrar
)

# Mostrar resultados en un notebook
exp.show_in_notebook()

# Guardar la explicación en un archivo HTML
exp.save_to_file('lime_explanation.html')

# Mostrar explicación en consola
for label in [0, 1]:
    print(f"\nEtiqueta {class_names[label]}:")

In [15]:
torch.cuda.empty_cache()
gc.collect()

10780

In [22]:
import os
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from lime.lime_text import LimeTextExplainer
import os
import gc
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from lime.lime_text import LimeTextExplainer

# Configurar el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()  # Liberar memoria

# Cargar el modelo y el tokenizador
nombredecarpetaatrabajar = "IMDB/"
output_dir = nombredecarpetaatrabajar + "/stopwords/" + "/model"
loaded_tokenizer = BertTokenizer.from_pretrained(output_dir)
loaded_model = BertForSequenceClassification.from_pretrained(output_dir).to(device)

# Función para predecir el sentimiento
def predict_sentiment(text):
    inputs = loaded_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)  # Reduce max_length
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = loaded_model(**inputs)
    logits = outputs.logits
    return torch.argmax(logits, dim=-1).item()

# Función para devolver probabilidades para LIME
# Función para devolver probabilidades corregida
def predict_proba(texts):
    batch_size = 8  # Tamaño pequeño para evitar saturar la GPU
    probabilities = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        inputs = loaded_tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = loaded_model(**inputs)
        batch_probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        probabilities.extend(batch_probs)
    return np.array(probabilities)  # Convertimos a un numpy array

# Ejemplo de texto
text_example ="Originalmente, yo era fanático de Tenacious D por su primer álbum y, naturalmente, escuché algunas canciones de The P.O.D., pero quedé bastante decepcionado. Después de ver la película, mi perspectiva cambió. La película es bastante divertida de principio a fin, y me encontré enganchado a pesar de que la trama era realmente absurda, gracias a las actitudes que KG y Jaybles retratan en la película. Mucho más entretenida y agradable que otras películas que he visto en el cine últimamente, como Saw III (aburrida y lenta) o Casino Royale (demasiado homoerótica), aunque he disfrutado mucho entregas anteriores. Si disfrutaste Borat, seguramente disfrutarás la historia de The Greatest Band on Earth."
predicted_class = predict_sentiment(text_example)
print(f"Clase predicha: {predicted_class}")

# Explicación con LIME
class_names = ["Negativo", "Positivo"]
explainer = LimeTextExplainer(class_names=class_names)

# Generar la explicación
exp = explainer.explain_instance(text_example, predict_proba, num_features=10)
exp.save_to_file("lime_explanationlmm.html")


Clase predicha: 1


In [20]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import torch.nn.functional as F
import spacy

# Cargar el modelo en español
nlp = spacy.load("es_core_news_sm")

frase = "¡Hola! Esto es un ejemplo con spaCy."




# Cargar el modelo y el tokenizador preentrenados
output_dir = nombredecarpetaatrabajar + "/stopwords/" + "/model"
loaded_tokenizer = BertTokenizer.from_pretrained(output_dir)
loaded_model = BertForSequenceClassification.from_pretrained(output_dir)

# Verificar si CUDA está disponible y mover el modelo a la GPU si es posible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)
loaded_model.eval()  # Colocar en modo evaluación

# Función para obtener la predicción de una oración
def predict_sentiment(text):
    inputs = loaded_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Mover los inputs a la GPU
    with torch.no_grad():  # Desactivar el cálculo de gradientes
        outputs = loaded_model(**inputs)
    logits = outputs.logits
    print(logits)
    probabilities = F.softmax(logits, dim=-1).cpu().numpy()
    print(probabilities)
     
    return probabilities[0]

# Función para analizar el impacto de cada palabra en la clasificación
def analyze_word_importance(text):
    # Obtener la predicción de la oración original
    original_probs = predict_sentiment(text)
    original_class = np.argmax(original_probs)
    print(f"Texto original: '{text}'")
    print(f"Predicción original: Clase {original_class}, Probabilidades {original_probs}")

    # Tokenizar el texto y convertirlo en tokens individuales
    doc = nlp(text)
    tokens = [token.text for token in doc]
    word_importance = []

    # Perturbar el texto eliminando una palabra a la vez
    for i, token in enumerate(tokens):
        # Crear un texto nuevo sin el token actual
        perturbed_tokens = tokens[:i] + tokens[i+1:]
        perturbed_text = loaded_tokenizer.convert_tokens_to_string(perturbed_tokens)

        # Obtener la predicción del texto perturbado
        perturbed_probs = predict_sentiment(perturbed_text)
        print(perturbed_probs)
        perturbed_class = np.argmax(perturbed_probs)

        # Calcular el cambio en la probabilidad para la clase original
        prob_change = original_probs[original_class] - perturbed_probs[original_class]
        
        # Guardar el resultado
        word_importance.append((token, prob_change, perturbed_class))
        print(f"Sin '{token}': Clase {perturbed_class}, Probabilidades {perturbed_probs}, Cambio en probabilidad de clase original: {prob_change}")

    # Ordenar las palabras por importancia en la probabilidad de la clase original
    word_importance = sorted(word_importance, key=lambda x: abs(x[1]), reverse=True)
    print("\nPalabras ordenadas por impacto en la predicción de la clase original:")
    for token, change, new_class in word_importance:
        print(f"Palabra '{token}': Cambio en probabilidad de clase original {change:.8f}, Nueva clase: {new_class}")

# Ejemplo de uso
text_example = "Originalmente, yo era fanático de Tenacious D por su primer álbum y, naturalmente, escuché algunas canciones de The P.O.D., pero quedé bastante decepcionado. Después de ver la película, mi perspectiva cambió. La película es bastante divertida de principio a fin, y me encontré enganchado a pesar de que la trama era realmente absurda, gracias a las actitudes que KG y Jaybles retratan en la película. Mucho más entretenida y agradable que otras películas que he visto en el cine últimamente, como Saw III (aburrida y lenta) o Casino Royale (demasiado homoerótica), aunque he disfrutado mucho entregas anteriores. Si disfrutaste Borat, seguramente disfrutarás la historia de The Greatest Band on Earth."
analyze_word_importance(text_example)

tensor([[-3.7104,  4.2124]], device='cuda:0')
[[3.622639e-04 9.996377e-01]]
Texto original: 'Originalmente, yo era fanático de Tenacious D por su primer álbum y, naturalmente, escuché algunas canciones de The P.O.D., pero quedé bastante decepcionado. Después de ver la película, mi perspectiva cambió. La película es bastante divertida de principio a fin, y me encontré enganchado a pesar de que la trama era realmente absurda, gracias a las actitudes que KG y Jaybles retratan en la película. Mucho más entretenida y agradable que otras películas que he visto en el cine últimamente, como Saw III (aburrida y lenta) o Casino Royale (demasiado homoerótica), aunque he disfrutado mucho entregas anteriores. Si disfrutaste Borat, seguramente disfrutarás la historia de The Greatest Band on Earth.'
Predicción original: Clase 1, Probabilidades [3.622639e-04 9.996377e-01]
tensor([[-3.6969,  4.1810]], device='cuda:0')
[[3.7889535e-04 9.9962103e-01]]
[3.7889535e-04 9.9962103e-01]
Sin 'Originalmente': Cl